In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from datetime import datetime

### scrapping
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS \
          X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) \
          Chrome/71.0.3578.98 Safari/537.36", \
          "Accept":"text/html,application/xhtml+xml,application/xml; \
          q=0.9,image/webp,image/apng,*/*;q=0.8"}

In [3]:
r = requests.get('https://www.imapi.org/perfil/osasco-sp', headers=my_headers)

In [4]:
content = r.content

In [5]:
soup = BeautifulSoup(content)

In [17]:
labels = []
values = []
dims = []


In [7]:
for d in soup.find_all('article', attrs={'class':'dimensao-sticky'}):
    for d1 in d.find_all('div', attrs={'class':'col-xs-3'}):
        label = d1.find('p', attrs={'class':'label'})
        dado = d1.find('p', attrs={'class':'dado'})
        print('Label: ', label.text)
        print('Dado: ', dado.text)
        print('------')

Label:  Região
Dado:  Sudeste
------
Label:  Porte da cidade
Dado:  Grande porte
------
Label:  Habitantes
Dado:  696.382
------
Label:  Nascimentos ao ano/ 1000 habitantes
Dado:  10.320
------


In [8]:
for d in soup.find_all('g'):
    if d.has_attr('data-nota'):
        print('Label: ', d['data-titulo'])
        print('Valor', d['data-nota'])

Label:  saúde
Valor 46
Label:  nutrição
Valor 45
Label:  cuidado responsivo
Valor 0
Label:  aprendizagem inicial
Valor 74
Label:  segurança e proteção
Valor 13


In [9]:
for d in soup.find_all('section', attrs={'class':'row bottom-xs'}):
    for d1 in d.find_all('div', attrs={'class': 'col-xs-4 tl'}):
        dado = d1.find('p', attrs={'class':'dado'})
        print('Dado', dado.text)
        label = d1.find('p', attrs={'class':'label'})
        print('Label:', label.text)

Dado 2.619º lugar de 5.570
Label: Ranking Brasil


In [18]:
for s in soup.find_all('section', attrs={'class':'card'}):
    for d in s.find_all('article', attrs={'class':'indicador'}):
        dado = d.find('p', attrs={'class':'dado'})
        print('Dado: ', dado.text)
        
        label = d.find('p', attrs={'class':'label'})
        print('Label', label.text)
        
        municipio = d.find('div', attrs={'class':'local ativo'})
        if municipio != None and municipio.has_attr('data-nota'):
            valor = municipio['data-nota']
            print('Valor:', valor)
        else:
            valor = 'N/A'
            print(valor)
            print('--------')
        
        dims.append(s['data-dimensao'])
        labels.append(label.text)
        values.append(valor)

Dado:  Início adequado do pré-natal
Label Percentual de gestantes que iniciaram o pré-natal com 12 semanas ou menos
Valor: 85,63
Dado:  Consultas de pré-natal
Label Percentual de gestantes com 6 ou mais consultas de pré‐natal 
Valor: 75,62
Dado:  Gravidez na adolescência
Label Percentual  de gravidez na adolescência
Valor: 13,38
Dado:  Cesáreas
Label Percentual de cesáreas
Valor: 56,61
Dado:  Prematuridade
Label Percentual de crianças nascidas com menos de 37 semanas completas de gestação
Valor: 13,30
Dado:  Baixo peso ao nascer
Label Percentual de nascidos vivos com menos de 2500g
Valor: 9,62
Dado:  Sífilis Congênita
Label Casos confirmados e notificados de sífilis congênita para cada 10.000 crianças  menores de 5 anos
Valor: 14,40
Dado:  Visitas domiciliares nos primeiros 10 dias de vida
Label Percentual de equipes da atenção básica que participaram do PMAQ 2º ciclo e fizeram visitas domiciliares nos primeiros 10 dias de vida 
Valor: 40,00
Dado:  Cobertura de imunização
Label Percent

In [19]:
res = {'dimensao': dims, 'label': labels, 'valor': values}
res

{'dimensao': ['saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'saude',
  'nutricao',
  'nutricao',
  'nutricao',
  'nutricao',
  'cuidado',
  'aprendizagem',
  'aprendizagem',
  'aprendizagem',
  'aprendizagem',
  'aprendizagem',
  'aprendizagem',
  'aprendizagem',
  'seguranca',
  'seguranca',
  'seguranca',
  'seguranca',
  'seguranca'],
 'label': ['Percentual de gestantes que iniciaram o pré-natal com 12 semanas ou menos',
  'Percentual de gestantes com 6 ou mais consultas de pré‐natal ',
  'Percentual  de gravidez na adolescência',
  'Percentual de cesáreas',
  'Percentual de crianças nascidas com menos de 37 semanas completas de gestação',
  'Percentual de nascidos vivos com menos de 2500g',
  'Casos confirmados e notificados de sífilis congênita para cada 10.000 crianças  menores de 5 anos',
  'Percentual de equipes da atenção básica que participaram do PMAQ 2º ciclo e fizeram visitas d

In [20]:
resultaso_osasco = pd.DataFrame().from_dict(res)

In [22]:
resultaso_osasco

,dimensao,label,valor
0,saude,Percentual de gestantes que iniciaram o pré-na...,"85,63"
1,saude,Percentual de gestantes com 6 ou mais consulta...,"75,62"
2,saude,Percentual de gravidez na adolescência,"13,38"
3,saude,Percentual de cesáreas,"56,61"
4,saude,Percentual de crianças nascidas com menos de 3...,"13,30"
5,saude,Percentual de nascidos vivos com menos de 2500g,"9,62"
6,saude,Casos confirmados e notificados de sífilis con...,"14,40"
7,saude,Percentual de equipes da atenção básica que pa...,"40,00"
8,saude,Percentual de crianças que receberam o primeir...,"76,79"
9,saude,Percentual da população coberta pela Atenção B...,"48,00"


### selenium - Paginas dinamicas

In [30]:
!pip install selenium
!pip install chromedriver

In [25]:
from selenium import webdriver
from bs4 import BeautifulSoup

url = 'https://www.imapi.org/perfil/osasco-sp'

In [26]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [37]:
wd = webdriver.Chrome(executable_path='/Users/dgoalmeida/Documents/datascience/chrome-driver/chromedriver', options=chrome_options)

In [38]:
wd.get(url)

In [39]:
html = wd.page_source

In [40]:
soup2 = BeautifulSoup(html,'html.parser')

In [46]:
municipios = [option.text for option in soup2.select('#municipios option')]

In [47]:
municipios

['abadia_de_goias-go',
 'abadia_dos_dourados-mg',
 'abadiania-go',
 'abaetetuba-pa',
 'abaete-mg',
 'abaiara-ce',
 'abare-ba',
 'abatia-pr',
 'abaira-ba',
 'abdon_batista-sc',
 'abel_figueiredo-pa',
 'abelardo_luz-sc',
 'abre_campo-mg',
 'abreu_e_lima-pe',
 'abreulandia-to',
 'acaiaca-mg',
 'acajutiba-ba',
 'acarape-ce',
 'acarau-ce',
 'acari-rn',
 'acara-pa',
 'acaua-pi',
 'acegua-rs',
 'acopiara-ce',
 'acorizal-mt',
 'acrelandia-ac',
 'acreuna-go',
 'adamantina-sp',
 'adelandia-go',
 'adolfo-sp',
 'adrianopolis-pr',
 'adustina-ba',
 'afogados_da_ingazeira-pe',
 'afonso_bezerra-rn',
 'afonso_claudio-es',
 'afonso_cunha-ma',
 'afranio-pe',
 'afua-pa',
 'agrestina-pe',
 'agricolandia-pi',
 'agrolandia-sc',
 'agronomica-sc',
 'aguanil-mg',
 'aguai-sp',
 'agudo-rs',
 'agudos_do_sul-pr',
 'agudos-sp',
 'aguiar-pb',
 'aguiarnopolis-to',
 'aimores-mg',
 'aiquara-ba',
 'aiuaba-ce',
 'aiuruoca-mg',
 'ajuricaba-rs',
 'alagoa_grande-pb',
 'alagoa_nova-pb',
 'alagoa-mg',
 'alagoinha_do_piaui-pi',

In [49]:
from time import sleep
from random import randint

In [57]:
labels = []
values = []
dims = []
munis = []
estados = []

In [58]:
i=1
for muni in municipios[:10]:
    print(i,sep=' ', end=' ', flush=True)
    i+=1
    r = requests.get('https://www.imapi.org/perfil/'+muni, headers = my_headers)
    content = r.content
    sleep(randint(2,4))
    soup = BeautifulSoup(content)
    
    for s in soup.find_all('section', attrs={'class':'card'}):
        for d in s.find_all('article', attrs={'class':'indicador'}):
            dado = d.find('p', attrs={'class':'dado'})
            label = d.find('p', attrs={'class':'label'})

            municipio = d.find('div', attrs={'class':'local ativo'})
            if municipio != None and municipio.has_attr('data-nota'):
                valor = municipio['data-nota']
            else:
                valor = np.NaN
            munis.append(muni.split('-')[0])
            estados.append(muni.split('-')[1])
            dims.append(s['data-dimensao'])
            labels.append(label.text)
            values.append(valor)

1 2 3 4 5 6 7 8 9 10 

In [60]:
res = {'estado': estados, 'municipio': munis, 'dimensao': dims, 'label': labels, 'valor': values}

In [61]:
resdf = pd.DataFrame.from_dict(res)

In [62]:
resdf

,estado,municipio,dimensao,label,valor
0,go,abadia_de_goias,saude,Percentual de gestantes que iniciaram o pré-na...,"87,30"
1,go,abadia_de_goias,saude,Percentual de gestantes com 6 ou mais consulta...,"81,50"
2,go,abadia_de_goias,saude,Percentual de gravidez na adolescência,"21,50"
3,go,abadia_de_goias,saude,Percentual de cesáreas,"69,00"
4,go,abadia_de_goias,saude,Percentual de crianças nascidas com menos de 3...,"14,00"
...,...,...,...,...,...
305,sc,abdon_batista,seguranca,Casos notificados de violência para cada 1000...,"6,13"
306,sc,abdon_batista,seguranca,Casos notificados de violência para cada 1000...,"2,87"
307,sc,abdon_batista,seguranca,Percentual de famílias beneficiárias do Progra...,"54,32"
308,sc,abdon_batista,seguranca,"Estimativa da concentração diária de PM2,5 (ug...","4,84"


In [63]:
resdf.to_csv('imapi.csv')

In [64]:
resdf.describe()

,estado,municipio,dimensao,label,valor
count,310,310,310,310,307
unique,7,10,5,31,217
top,ba,abaetetuba,saude,Mortes de mulheres por causas relacionadas à g...,"0,00"
freq,62,31,140,10,50
